In [ ]:
import numpy as np
np.array = np.asarray  # ⚠️ Fixes NumPy 2.0 compatibility issue


In [ ]:
!pip install sympy>=1.13.3

In [ ]:
import sympy
import mpmath
print(f"SymPy version: {sympy.__version__}")
print(f"mpmath version: {mpmath.__version__}")
print("Success - packages imported correctly!")

SymPy version: 1.13.1
mpmath version: 1.3.0
Success - packages imported correctly!


In [ ]:
!pip install transformers datasets nltk scikit-learn pandas
print("Success")

Success


In [ ]:
import numpy as np
import pandas as pd
print("✓ Both packages working!")
print(f"NumPy version: {np.__version__}")
print(f"Pandas version: {pd.__version__}")
print(f"NumPy location: {np.__file__}")
print(f"Pandas location: {pd.__file__}")

✓ Both packages working!
NumPy version: 1.26.4
Pandas version: 2.2.2
NumPy location: /usr/local/lib/python3.11/dist-packages/numpy/__init__.py
Pandas location: /usr/local/lib/python3.11/dist-packages/pandas/__init__.py


In [ ]:
import pandas as pd
import numpy as np
import torch
from datasets import Dataset
from transformers import (
    AutoTokenizer,
    AutoModelForSequenceClassification,
    Trainer,
    TrainingArguments,
    DataCollatorWithPadding,
)
from sklearn.metrics import accuracy_score, precision_recall_fscore_support

print("✅ All libraries imported successfully!")



✅ All libraries imported successfully!


In [ ]:
import numpy as np
np.array = np.asarray  # patch to fix numpy copy=False error with datasets
# Load Tamil training and validation datasets
tamil_train = pd.read_csv("tamil_offensive_speech_train.csv")[["comment", "label"]]
tamil_val = pd.read_csv("tamil_offensive_speech_val.csv")[["comment", "label"]]

# Rename 'comment' to 'text' (standardized)
tamil_train = tamil_train.rename(columns={"comment": "text"})
tamil_val = tamil_val.rename(columns={"comment": "text"})

# Check data shape and first few rows
print(f"Train shape: {tamil_train.shape}")
print(f"Validation shape: {tamil_val.shape}")
tamil_train.head()


Train shape: (27875, 2)
Validation shape: (6969, 2)


,text,label
0,omg that bgm make me goosebumb...,0
1,neraya neraya neraya neraya neraya neraya.,0
2,thalaivar mersal look .semma massss thalaiva ....,0
3,paaaa... repeat mode.... adra adra adraaaaa......,0
4,epaa ena panaporam... sweet sapade poram... aw...,0


In [ ]:
import re

# Function to clean text
def clean_text(text):
    if not isinstance(text, str):
        return ""
    text = text.strip()               # Remove leading/trailing spaces
    text = text.lower()               # Lowercase all text
    text = re.sub(r'\s+', ' ', text) # Replace multiple spaces/newlines with single space
    # Optional: Remove special characters except Tamil letters, numbers and basic punctuation
    # text = re.sub(r"[^a-zA-Z0-9அஆஇஈஉஊஎஏஐஒஓஔகஙசஞடணதநபமயரலவஷஸஹாிீுூெேைொோௌ்.,!?]", " ", text)
    return text

# Apply cleaning to train and val data
tamil_train['text'] = tamil_train['text'].apply(clean_text)
tamil_val['text'] = tamil_val['text'].apply(clean_text)

# Remove empty or whitespace-only rows after cleaning
tamil_train = tamil_train[tamil_train['text'].str.strip() != ""]
tamil_val = tamil_val[tamil_val['text'].str.strip() != ""]

# Reset index after cleaning
tamil_train = tamil_train.reset_index(drop=True)
tamil_val = tamil_val.reset_index(drop=True)

print(f"Cleaned train shape: {tamil_train.shape}")
print(f"Cleaned val shape: {tamil_val.shape}")


Cleaned train shape: (27870, 2)
Cleaned val shape: (6969, 2)


In [ ]:
print(tamil_train['label'].dtype)  # Should output: int64 or int32
print(tamil_val['label'].dtype)    # Should output: int64 or int32


int64
int64


In [ ]:
from transformers import AutoTokenizer

# Load tokenizer
tokenizer = AutoTokenizer.from_pretrained("xlm-roberta-base")

# Tokenization function
def tokenize_function(examples):
    return tokenizer(examples['text'], truncation=True, padding='max_length', max_length=256)

# Convert pandas DataFrame to Huggingface Dataset first
from datasets import Dataset

train_dataset = Dataset.from_pandas(tamil_train)
val_dataset = Dataset.from_pandas(tamil_val)

# Apply tokenization (batched=True for faster processing)
train_dataset = train_dataset.map(tokenize_function, batched=True)
val_dataset = val_dataset.map(tokenize_function, batched=True)

# Set the format for PyTorch tensors (required for training)
train_dataset.set_format(type='torch', columns=['input_ids', 'attention_mask', 'label'])
val_dataset.set_format(type='torch', columns=['input_ids', 'attention_mask', 'label'])

print("Tokenization done!")
# Temporarily disable the tensor format to access raw data safely
train_dataset.set_format(type=None)

# Now you can safely access the first example as a dict with numpy arrays or lists
example = train_dataset[0]
print({k: type(v) for k, v in example.items()})
print(example)  # See full example if you want

# After inspection, set the format back for training
train_dataset.set_format(type='torch', columns=['input_ids', 'attention_mask', 'label'])



/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


Map:   0%|          | 0/27870 [00:00<?, ? examples/s]

Map:   0%|          | 0/6969 [00:00<?, ? examples/s]

Tokenization done!
{'text': <class 'str'>, 'label': <class 'int'>, 'input_ids': <class 'list'>, 'attention_mask': <class 'list'>}
{'text': 'omg that bgm make me goosebumb...', 'label': 0, 'input_ids': [0, 171, 177, 450, 6, 11821, 39, 3249, 163, 738, 8364, 978, 6492, 27, 2, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1], 'a

In [ ]:
from transformers import AutoModelForSequenceClassification

# Number of classes (labels)
num_labels = 2  # since your labels are already integers 0 or 1

# Load the pretrained XLM-RoBERTa model for sequence classification
model = AutoModelForSequenceClassification.from_pretrained("xlm-roberta-base", num_labels=num_labels)

print("Model loaded with", num_labels, "labels.")

Some weights of XLMRobertaForSequenceClassification were not initialized from the model checkpoint at xlm-roberta-base and are newly initialized: ['classifier.dense.bias', 'classifier.dense.weight', 'classifier.out_proj.bias', 'classifier.out_proj.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Model loaded with 2 labels.


In [ ]:
import numpy as np
from sklearn.metrics import precision_recall_fscore_support, accuracy_score

def compute_metrics(eval_pred):
    logits, labels = eval_pred
    predictions = np.argmax(logits, axis=1)

    precision, recall, f1, _ = precision_recall_fscore_support(labels, predictions, average='binary', zero_division=0)
    accuracy = accuracy_score(labels, predictions)

    return {
        "accuracy": accuracy,
        "precision": precision,
        "recall": recall,
        "f1": f1
    }

In [ ]:
import transformers
from transformers import TrainingArguments

print("Transformers version:", transformers.__version__)
print("TrainingArguments module:", TrainingArguments.__module__)
print("Transformers installed at:", transformers.__file__)


Transformers version: 4.52.4
TrainingArguments module: transformers.training_args
Transformers installed at: /usr/local/lib/python3.11/dist-packages/transformers/__init__.py


In [ ]:
!pip show transformers
!pip list | grep transformers


Name: transformers
Version: 4.52.4
Summary: State-of-the-art Machine Learning for JAX, PyTorch and TensorFlow
Home-page: https://github.com/huggingface/transformers
Author: The Hugging Face team (past and future) with the help of all our contributors (https://github.com/huggingface/transformers/graphs/contributors)
Author-email: transformers@huggingface.co
License: Apache 2.0 License
Location: /usr/local/lib/python3.11/dist-packages
Requires: filelock, huggingface-hub, numpy, packaging, pyyaml, regex, requests, safetensors, tokenizers, tqdm
Required-by: peft, sentence-transformers
sentence-transformers                 4.1.0
transformers                          4.52.4


In [ ]:
!pip install --upgrade transformers==4.52.4


In [ ]:
!pip show transformers


Name: transformers
Version: 4.52.4
Summary: State-of-the-art Machine Learning for JAX, PyTorch and TensorFlow
Home-page: https://github.com/huggingface/transformers
Author: The Hugging Face team (past and future) with the help of all our contributors (https://github.com/huggingface/transformers/graphs/contributors)
Author-email: transformers@huggingface.co
License: Apache 2.0 License
Location: /usr/local/lib/python3.11/dist-packages
Requires: filelock, huggingface-hub, numpy, packaging, pyyaml, regex, requests, safetensors, tokenizers, tqdm
Required-by: peft, sentence-transformers


In [ ]:
import pandas as pd
from datasets import Dataset
from transformers import AutoTokenizer, AutoModelForSequenceClassification
from transformers import TrainingArguments, Trainer, DataCollatorWithPadding
import numpy as np
np.array = np.asarray  # patch to fix numpy copy=False error with datasets

from sklearn.metrics import f1_score

# Load CSV files
tamil_train = pd.read_csv("tamil_offensive_speech_train.csv")
tamil_val = pd.read_csv("tamil_offensive_speech_val.csv")

# Replace NaN or None with empty string or drop rows
tamil_train['comment'] = tamil_train['comment'].fillna("").astype(str)
tamil_val['comment'] = tamil_val['comment'].fillna("").astype(str)
tamil_train = tamil_train.rename(columns={"comment": "text"})
tamil_val = tamil_val.rename(columns={"comment": "text"})


# Convert to HuggingFace Datasets
train_dataset = Dataset.from_pandas(tamil_train)
val_dataset = Dataset.from_pandas(tamil_val)

model_name = "xlm-roberta-base"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForSequenceClassification.from_pretrained(model_name, num_labels=2)

def tokenize_function(examples):
    # examples["text"] should be a list of strings if batched=True,
    # or a single string if batched=False
    # So just pass it directly
    return tokenizer(
        examples["text"],  # list or str both work for tokenizer
        padding="max_length",
        truncation=True,
        max_length=128,
            )

# Tokenize datasets
train_dataset = train_dataset.map(tokenize_function, batched=True)
val_dataset = val_dataset.map(tokenize_function, batched=True)

# Rename label column to labels
train_dataset = train_dataset.rename_column("label", "labels")
val_dataset = val_dataset.rename_column("label", "labels")

# Set format for PyTorch
train_dataset.set_format(type="torch", columns=["input_ids", "attention_mask", "labels"])
val_dataset.set_format(type="torch", columns=["input_ids", "attention_mask", "labels"])

# Data collator for dynamic padding
data_collator = DataCollatorWithPadding(tokenizer=tokenizer)

# Metrics function
def compute_metrics(eval_pred):
    logits, labels = eval_pred
    preds = np.argmax(logits, axis=-1)
    f1 = f1_score(labels, preds, average="weighted")
    return {"f1": f1}

# Training arguments - keep batch size small to speed up training
training_args = TrainingArguments(
    output_dir="./model_checkpoints",
    eval_strategy="epoch",
    save_strategy="epoch",
    learning_rate=2e-5,
    per_device_train_batch_size=8,
    per_device_eval_batch_size=8,
    num_train_epochs=2,
    weight_decay=0.01,
    metric_for_best_model="f1",
    greater_is_better=True,
    logging_dir="./logs",
    logging_steps=10,
    save_total_limit=2,
    seed=42,
)

# Initialize Trainer
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=val_dataset,
    tokenizer=tokenizer,
    data_collator=data_collator,
    compute_metrics=compute_metrics,
)

# Train and evaluate
trainer.train()
results = trainer.evaluate()
print("Evaluation results:", results)

# Save model and tokenizer
trainer.save_model("./saved_model")
tokenizer.save_pretrained("./saved_model")
print("Model and tokenizer saved in './saved_model'")


Some weights of XLMRobertaForSequenceClassification were not initialized from the model checkpoint at xlm-roberta-base and are newly initialized: ['classifier.dense.bias', 'classifier.dense.weight', 'classifier.out_proj.bias', 'classifier.out_proj.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Map:   0%|          | 0/27875 [00:00<?, ? examples/s]

Map:   0%|          | 0/6969 [00:00<?, ? examples/s]

<ipython-input-12-2786844690>:81: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(
wandb: WARNING The `run_name` is currently set to the same value as `TrainingArguments.output_dir`. If this was not intended, please specify a different run name by setting the `TrainingArguments.run_name` parameter.
wandb: Currently logged in as: mathumethaseenivasagan (mathumethaseenivasagan-panimalar-engineering-college) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin


Epoch,Training Loss,Validation Loss,F1
1,0.464500,0.401100,0.807253
2,0.326900,0.394778,0.829011


Evaluation results: {'eval_loss': 0.3947780132293701, 'eval_f1': 0.8290107170325807, 'eval_runtime': 47.1609, 'eval_samples_per_second': 147.771, 'eval_steps_per_second': 18.49, 'epoch': 2.0}
Model and tokenizer saved in './saved_model'


In [ ]:
!pip install matplotlib seaborn scikit-learn


In [ ]:
from sklearn.metrics import confusion_matrix
import numpy as np

predictions = trainer.predict(val_dataset)
y_true = predictions.label_ids
y_pred = np.argmax(predictions.predictions, axis=1)

cm = confusion_matrix(y_true, y_pred)
labels = ['non-hate', 'hate']

print("Confusion Matrix:")
print(f"           Predicted")
print(f"Actual  {labels[0]:>8} {labels[1]:>8}")
for i, label in enumerate(labels):
    print(f"{label:>6}  {cm[i,0]:8d} {cm[i,1]:8d}")

Confusion Matrix:
           Predicted
Actual  non-hate     hate
non-hate      4825      460
  hate       700      984


In [ ]:
# Your confusion matrix results
tn, fp, fn, tp = 4825, 460, 700, 984

print("=" * 60)
print("     HATE SPEECH DETECTION MODEL - PERFORMANCE ANALYSIS")
print("=" * 60)
print()
print("Confusion Matrix:")
print("                    PREDICTED")
print("                non-hate    hate    Total")
print("ACTUAL non-hate    {:4d}     {:4d}     {:4d}".format(tn, fp, tn+fp))
print("       hate        {:4d}     {:4d}     {:4d}".format(fn, tp, fn+tp))
print("       Total       {:4d}     {:4d}     {:4d}".format(tn+fn, fp+tp, tn+fp+fn+tp))
print()

# Calculate basic metrics
total_samples = tn + fp + fn + tp
accuracy = (tp + tn) / total_samples
error_rate = (fp + fn) / total_samples

print("OVERALL PERFORMANCE:")
print(f"Total Samples: {total_samples:,}")
print(f"Accuracy: {accuracy:.4f} ({tp+tn:,}/{total_samples:,}) - {accuracy*100:.2f}%")
print(f"Error Rate: {error_rate:.4f} ({fp+fn:,}/{total_samples:,}) - {error_rate*100:.2f}%")
print()

# Per-class metrics
# Non-hate class
precision_non_hate = tn / (tn + fn)
recall_non_hate = tn / (tn + fp)
f1_non_hate = 2 * (precision_non_hate * recall_non_hate) / (precision_non_hate + recall_non_hate)

# Hate class
precision_hate = tp / (tp + fp)
recall_hate = tp / (tp + fn)
f1_hate = 2 * (precision_hate * recall_hate) / (precision_hate + recall_hate)

print("DETAILED METRICS BY CLASS:")
print("-" * 50)
print("NON-HATE CLASS:")
print(f"  Precision: {precision_non_hate:.4f} - Of all predicted non-hate, {precision_non_hate*100:.1f}% were correct")
print(f"  Recall:    {recall_non_hate:.4f} - Of all actual non-hate, {recall_non_hate*100:.1f}% were caught")
print(f"  F1-Score:  {f1_non_hate:.4f}")
print(f"  Support:   {tn + fp:,} samples")
print()

print("HATE SPEECH CLASS:")
print(f"  Precision: {precision_hate:.4f} - Of all predicted hate, {precision_hate*100:.1f}% were correct")
print(f"  Recall:    {recall_hate:.4f} - Of all actual hate, {recall_hate*100:.1f}% were caught")
print(f"  F1-Score:  {f1_hate:.4f}")
print(f"  Support:   {tp + fn:,} samples")
print()

# Error analysis
fpr = fp / (fp + tn)  # False Positive Rate
fnr = fn / (fn + tp)  # False Negative Rate

print("ERROR BREAKDOWN:")
print("-" * 50)
print(f"True Positives (TP):  {tp:,} - Correctly identified hate speech")
print(f"True Negatives (TN):  {tn:,} - Correctly identified non-hate content")
print(f"False Positives (FP): {fp:,} - Non-hate content flagged as hate")
print(f"False Negatives (FN): {fn:,} - Hate speech that was missed")
print()
print(f"False Positive Rate: {fpr:.4f} ({fpr*100:.2f}%)")
print(f"  → {fp:,} out of {fp+tn:,} non-hate posts were incorrectly flagged")
print(f"False Negative Rate: {fnr:.4f} ({fnr*100:.2f}%)")
print(f"  → {fn:,} out of {fn+tp:,} hate speech posts were missed")
print()

# Balanced metrics
macro_precision = (precision_non_hate + precision_hate) / 2
macro_recall = (recall_non_hate + recall_hate) / 2
macro_f1 = (f1_non_hate + f1_hate) / 2

weighted_precision = (precision_non_hate * (tn+fp) + precision_hate * (tp+fn)) / total_samples
weighted_recall = (recall_non_hate * (tn+fp) + recall_hate * (tp+fn)) / total_samples
weighted_f1 = (f1_non_hate * (tn+fp) + f1_hate * (tp+fn)) / total_samples

print("AVERAGE METRICS:")
print("-" * 50)
print(f"Macro Average:")
print(f"  Precision: {macro_precision:.4f}")
print(f"  Recall:    {macro_recall:.4f}")
print(f"  F1-Score:  {macro_f1:.4f}")
print()
print(f"Weighted Average:")
print(f"  Precision: {weighted_precision:.4f}")
print(f"  Recall:    {weighted_recall:.4f}")
print(f"  F1-Score:  {weighted_f1:.4f}")
print()

# Class distribution analysis
hate_percentage = (tp + fn) / total_samples * 100
non_hate_percentage = (tn + fp) / total_samples * 100

print("DATASET COMPOSITION:")
print("-" * 50)
print(f"Non-hate samples: {tn+fp:,} ({non_hate_percentage:.1f}%)")
print(f"Hate samples:     {tp+fn:,} ({hate_percentage:.1f}%)")
print(f"Class imbalance ratio: {(tn+fp)/(tp+fn):.1f}:1 (non-hate:hate)")
print()

# Performance interpretation
print("MODEL PERFORMANCE ASSESSMENT:")
print("-" * 50)

# Overall assessment
if accuracy > 0.85:
    accuracy_assessment = "Excellent"
elif accuracy > 0.75:
    accuracy_assessment = "Good"
elif accuracy > 0.65:
    accuracy_assessment = "Fair"
else:
    accuracy_assessment = "Poor"

print(f"Overall Accuracy: {accuracy_assessment} ({accuracy*100:.1f}%)")

# Hate detection specific
if precision_hate > 0.8:
    precision_assessment = "High precision - Low false positive rate"
elif precision_hate > 0.6:
    precision_assessment = "Moderate precision - Some false positives"
else:
    precision_assessment = "Low precision - Many false positives"

if recall_hate > 0.8:
    recall_assessment = "High recall - Catches most hate speech"
elif recall_hate > 0.6:
    recall_assessment = "Moderate recall - Misses some hate speech"
else:
    recall_assessment = "Low recall - Misses significant hate speech"

print(f"Hate Detection Precision: {precision_assessment} ({precision_hate:.3f})")
print(f"Hate Detection Recall: {recall_assessment} ({recall_hate:.3f})")
print()

# Recommendations
print("RECOMMENDATIONS:")
print("-" * 50)
if fnr > 0.3:
    print("⚠️  High False Negative Rate - Consider:")
    print("   • Adjusting classification threshold")
    print("   • Adding more hate speech examples to training")
    print("   • Using class weights to penalize false negatives more")

if fpr > 0.1:
    print("⚠️  High False Positive Rate - Consider:")
    print("   • Improving model's understanding of context")
    print("   • Adding more diverse non-hate examples")
    print("   • Fine-tuning to reduce over-sensitivity")

if f1_hate < 0.7:
    print("⚠️  Low F1-Score for hate detection - Consider:")
    print("   • Balancing precision and recall")
    print("   • More sophisticated model architecture")
    print("   • Better feature engineering")

print(f"\n✅ Model successfully identifies {recall_hate*100:.1f}% of hate speech")
print(f"✅ {precision_hate*100:.1f}% of hate predictions are correct")

     HATE SPEECH DETECTION MODEL - PERFORMANCE ANALYSIS

Confusion Matrix:
                    PREDICTED
                non-hate    hate    Total
ACTUAL non-hate    4825      460     5285
       hate         700      984     1684
       Total       5525     1444     6969

OVERALL PERFORMANCE:
Total Samples: 6,969
Accuracy: 0.8335 (5,809/6,969) - 83.35%
Error Rate: 0.1665 (1,160/6,969) - 16.65%

DETAILED METRICS BY CLASS:
--------------------------------------------------
NON-HATE CLASS:
  Precision: 0.8733 - Of all predicted non-hate, 87.3% were correct
  Recall:    0.9130 - Of all actual non-hate, 91.3% were caught
  F1-Score:  0.8927
  Support:   5,285 samples

HATE SPEECH CLASS:
  Precision: 0.6814 - Of all predicted hate, 68.1% were correct
  Recall:    0.5843 - Of all actual hate, 58.4% were caught
  F1-Score:  0.6292
  Support:   1,684 samples

ERROR BREAKDOWN:
--------------------------------------------------
True Positives (TP):  984 - Correctly identified hate speech
True Ne